In [1]:
from urllib.parse import quote
from requests_html import HTMLSession
import requests


class SubtitleDownloader:
    def __init__(self,verbose=False):
        self.response = None
        self.verbose = verbose
        
        
    def url_to_response(self, url):
        session = HTMLSession()
        self.response = session.get(url)

        if self.verbose == True:
            print(self.response)
        else:
            pass
        
        return self.response


    
    
sub_downloader = SubtitleDownloader()


    
    
    
    
    
    
def search_for_subtitle(query):
    website_url = "https://yts-subs.com/"
    query_encoded = quote(query)
    website_search_url = website_url + "search/" + query_encoded

    response = url_to_response(website_search_url)
    return response


def search_result_to_movie_info(single_search_result):
    poster = single_search_result.find('.media-object', first=True).attrs["src"]
    title = single_search_result.find(".media-heading", first=True).text

    year_raw = single_search_result.find(".movinfo-section")[0].text
    year = int(year_raw.split()[0])
    movie_page_raw = single_search_result.find(".media-body", first=True).find("a", first=True)
    movie_page = movie_page_raw.absolute_links.pop()

    movie_info = {"title":title,
                 "year": year,
                 "poster": poster,
                 "movie_page": movie_page}

    return movie_info


def select_movie_number(movie_info_list):
    for i in range(len(movie_info_list)):
        single_movie_info = movie_info_list[i]
        print(f'{i+1} : {single_movie_info["title"]} - {single_movie_info["year"]}')

    selected_movie = int(input("Enter the movie number: "))
    return movie_info_list[selected_movie - 1]
              

def subtitle_html_to_dict(single_subtitle):
    subtitle_language = single_subtitle.find(".sub-lang", first=True).text

    download_cell = single_subtitle.find('.download-cell', first=True)
    download_page_link = download_cell.find("a", first=True).absolute_links.pop()

    subtitle_rating = int(single_subtitle.find(".rating-cell", first=True).text)

    subtitle_dict = {"language": subtitle_language,
                    "download_page": download_page_link,
                    "rating": subtitle_rating}

    return subtitle_dict


def filter_subtitles(subtitle_info_list, filter_language="English"):
    results = []
    for single_subtitle in subtitle_info_list:
        if single_subtitle['language'] == filter_language:
            results.append(single_subtitle)

    if len(results) == 0:
        selected_subtitle = None
    else:
        selected_subtitle = results[0]

    return selected_subtitle


def download_page_to_zip_url(final_subtitle):
    response = url_to_response(final_subtitle["download_page"])
    download_link = response.html.find('.download-subtitle', first=True).attrs["href"]
    return download_link


def download_zip_file(zip_url):
    zip_file_data = requests.get(zip_url).content
    filename = zip_url.split("/")[-1]

    with open(filename, "wb") as zip_file:
        zip_file.write(zip_file_data)
    
    return 0


def main():
    query = input("Enter movie name: ") # take movie name from user
    response = search_for_subtitle(query) # search for movie on website
    search_results = response.html.find('.media-movie-clickable') # get all the results

    if len(search_results) > 0:
        print(f"found {len(search_results)} subtitles. ")
    else:
        print("No results found.")
        return 0 # exit is no result is found

    # take search results convert them into a dict
    movie_info_list = list(map(search_result_to_movie_info, search_results))

    # Ask user to select a movie from the list
    selected_movie = select_movie_number(movie_info_list)

    # get the selected movie page
    movie_page_response = url_to_response(selected_movie["movie_page"])

    # find the list of subtitles for that movie
    subtitle_list = movie_page_response.html.find('.high-rating')
 
    # convert html subtitles to dict
    subtitle_info_list = list(map(subtitle_html_to_dict, subtitle_list))
    
    # select subtitle based on language and rating
    final_subtitle = filter_subtitles(subtitle_info_list)
    
    # get the zip file url from the movie page
    zip_url = download_page_to_zip_url(final_subtitle)
    
    # download the zip file
    download_zip_file(zip_url)
    print("Downloaded Subtitle in the current Directory.")
          

main()

KeyboardInterrupt: 

In [36]:
from urllib.parse import quote
from requests_html import HTMLSession
import requests


class SubtitleDownloader:
    """
    An example use of this class.
    sub_downloader = SubtitleDownloader(verbose=True)


    sub_downloader.search_for_subtitle("fight club")
    sub_downloader.response_to_dict()
    sub_downloader.select_movie_number(movie_number=1)
    sub_downloader.get_subtitles_from_selected_movie()
    sub_downloader.download_subtitle_zip()
    """
    def __init__(self,verbose=False):
        self.response = None
        self.verbose = verbose
        
        
    def url_to_response(self, url):
        """takes a url, make a request and return the response"""
        
        session = HTMLSession()
        response = session.get(url)
        
        return response
    
    def search_for_subtitle(self, query):
        """Take a query string, make the url to search and get the response"""
        website_url = "https://yts-subs.com/"
        query_encoded = quote(query)
        website_search_url = website_url + "search/" + query_encoded

        self.response = self.url_to_response(website_search_url)
        
        if self.verbose == True:
            print(self.response)
        else:
            pass
        
        return self.response
    
    def search_result_to_movie_info(self, single_search_result):
        """Takes html of a search result, converts it into a dict
        and return the dict.
        """
        poster = single_search_result.find('.media-object', first=True).attrs["src"]
        title = single_search_result.find(".media-heading", first=True).text

        year_raw = single_search_result.find(".movinfo-section")[0].text
        year = int(year_raw.split()[0])
        movie_page_raw = single_search_result.find(".media-body", first=True).find("a", first=True)
        movie_page = movie_page_raw.absolute_links.pop()

        movie_info = {"title":title,
                     "year": year,
                     "poster": poster,
                     "movie_page": movie_page}

        return movie_info
    
    def response_to_dict(self):
        """Takes the reponse page, finds all the search results
        convert search results into dict and return the list of dicts."""
        search_results = self.response.html.find('.media-movie-clickable') # get all the results

        if len(search_results) > 0:
            if self.verbose == True:
                print(f'Found {len(search_results)} results.')
            else:
                pass
        
        else:
            print("No results found.")
            return 0 # exit is no result is found
        

        # take search results convert them into a dict
        movie_info_list = list(map(self.search_result_to_movie_info, search_results))
        
        if self.verbose == True:
            print(f'Parsed {len(movie_info_list)} results.')
        else:
            pass
        
        self.movie_info_list = movie_info_list
        
        return self.movie_info_list
        
        
    def select_movie_number(self, movie_number=None):
        """Let the user select a movie to download subtitle for. """

        if movie_number == None:
            for i in range(len(self.movie_info_list)):
                single_movie_info = self.movie_info_list[i]
                print(f'{i+1} : {single_movie_info["title"]} - {single_movie_info["year"]}')

            selected_movie = int(input("Enter the movie number: "))
        else:
            selected_movie = movie_number

        selected_movie_info  = self.movie_info_list[selected_movie - 1]
        self.selected_movie_info = selected_movie_info
        return self.selected_movie_info

    
    def subtitle_html_to_dict(self, single_subtitle):
        """Takes html for single subtitle result and convert it to dict then return it."""
        subtitle_language = single_subtitle.find(".sub-lang", first=True).text

        download_cell = single_subtitle.find('.download-cell', first=True)
        download_page_link = download_cell.find("a", first=True).absolute_links.pop()

        subtitle_rating = int(single_subtitle.find(".rating-cell", first=True).text)

        subtitle_dict = {"language": subtitle_language,
                        "download_page": download_page_link,
                        "rating": subtitle_rating}

        return subtitle_dict
                      
    def filter_subtitles(self, subtitle_info_list, filter_language="English"):
        """Takes all the subtitle dicts and filter them for language and takes the first one."""
        results = []
        for single_subtitle in subtitle_info_list:
            if single_subtitle['language'] == filter_language:
                results.append(single_subtitle)

        if len(results) == 0:
            selected_subtitle = None
        else:
            selected_subtitle = results[0]

        return selected_subtitle
                      
    def get_subtitles_from_selected_movie(self):
        """Find all the sub for selected movie and select the best one."""
        # get the selected movie page
        movie_page_response = self.url_to_response(self.selected_movie_info["movie_page"])

        # find the list of subtitles for that movie
        subtitle_list = movie_page_response.html.find('.high-rating')
        
        if self.verbose == True:
            print(f'Found {len(subtitle_list)} subtitles for selected movie.')
        else:
            pass


        # convert html subtitles to dict
        subtitle_info_list = list(map(self.subtitle_html_to_dict, subtitle_list))
                      
        # select subtitle based on language and rating
        final_subtitle = self.filter_subtitles(subtitle_info_list)
        self.final_subtitle = final_subtitle

    def download_page_to_zip_url(self):
        """Get the download link for the zip file from the download page."""
        response = self.url_to_response(self.final_subtitle["download_page"])
        download_link = response.html.find('.download-subtitle', first=True).attrs["href"]
        return download_link


    def download_zip_file(self, zip_url):
        """Downloads the zip file in the current directory."""
        zip_file_data = requests.get(zip_url).content
        filename = zip_url.split("/")[-1]

        with open(filename, "wb") as zip_file:
            zip_file.write(zip_file_data)

        return 0
                      
    def download_subtitle_zip(self):
        """get the zip url and download it."""
        
        # get the zip file url from the movie page
        zip_url = self.download_page_to_zip_url()

        # download the zip file
        self.download_zip_file(zip_url)
                      
        if self.verbose == True:
            print("Downloaded Subtitle in the current Directory.")
        else:
            pass
                      
        return 0
        
        
sub_downloader = SubtitleDownloader(verbose=True)


sub_downloader.search_for_subtitle("fight club")
sub_downloader.response_to_dict()
sub_downloader.select_movie_number(movie_number=1)
sub_downloader.get_subtitles_from_selected_movie()
sub_downloader.download_subtitle_zip()

<Response [200]>
Found 2 results.
Parsed 2 results.
Found 68 subtitles for selected movie.
Downloaded Subtitle in the current Directory.


0

In [37]:
help(sub_downloader)

Help on SubtitleDownloader in module __main__ object:

class SubtitleDownloader(builtins.object)
 |  An example use of this class.
 |  sub_downloader = SubtitleDownloader(verbose=True)
 |  
 |  
 |  sub_downloader.search_for_subtitle("fight club")
 |  sub_downloader.response_to_dict()
 |  sub_downloader.select_movie_number(movie_number=1)
 |  sub_downloader.get_subtitles_from_selected_movie()
 |  sub_downloader.download_subtitle_zip()
 |  
 |  Methods defined here:
 |  
 |  __init__(self, verbose=False)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  download_page_to_zip_url(self)
 |      Get the download link for the zip file from the download page.
 |  
 |  download_subtitle_zip(self)
 |      get the zip url and download it.
 |  
 |  download_zip_file(self, zip_url)
 |      Downloads the zip file in the current directory.
 |  
 |  filter_subtitles(self, subtitle_info_list, filter_language='English')
 |      Takes all the subtitle dicts and filter them 

In [45]:
class Fridge:
    def __init__(self):
        self.door_count = 0
        
    def open_door(self):
        print("Door opened.")
        self.door_count += 1
        
my_fridge = Fridge()

In [50]:
my_fridge.open_door()
print(my_fridge.door_count)

Door opened.
5


In [55]:
class BankAccount:
    def __init__(self, name):
        self.name = name
        self.balance = 0
        
    def credit(self, amount):
        self.balance += amount
        print(f"Added - {amount} \nCurrent Balance - {self.balance}")
        
    def debit(self, amount):
        if amount > self.balance:
            print("Not enough balance.")
        else:
            self.balance -= amount
            print(f"Withdrawn - {amount} \nCurrent Balance - {self.balance}")
        
    def check_balance(self):
        print(f"Current Balance - {self.balance}")
        
my_account = BankAccount("hasan")


In [56]:
my_account.credit(1000)

Added - 1000 
Current Balance - 1000


In [57]:
my_account.check_balance()

Current Balance - 1000


In [58]:
my_account.debit(345)

Withdrawn - 345 
Current Balance - 655


In [60]:
my_account.debit(655)

Withdrawn - 655 
Current Balance - 0
